# Progetto di Analisi Immagini e Video:

# Generazione di Volti con il Diffusion Model

Molinaro Pasquale 235309, Centraco Giuseppe 227591

# Indice

1. [Introduzione](#introduzione)
    1.1. [Librerie utilizzate](#librerie-utilizzate)
    1.2. [Configurazione ambiente](#configurazione-ambiente)
2. [Data Augmentation](#data-augmentation)
3. [Classe EMA](#classe-ema)
4. [Calcolo del passo di diffusione](#calcolo-del-passo-di-diffusione)
5. [Metodi ausiliari per la creazione della rete](#metodi-ausialiari-per-la-creazione-della-rete)
6. [Rete convoluzionale](#rete-convoluzionale)
7. [Training Loop](#training-loop)
8. [Main](#main)

# Introduzione <a id="introduzione"></a>

Il Diffusion Model è un potente algoritmo di deep learning che consente di generare immagini di alta qualità utilizzando una tecnica di diffusione stocastica. La generazione di volti realistici è una sfida affascinante nell'ambito dell'intelligenza artificiale. Esso si basa su un'architettura neurale generativa avanzata, in grado di apprendere e riprodurre le caratteristiche distintive dei volti umani con notevole fedeltà.

In questo notebook, esploreremo il processo di generazione di volti passo dopo passo.


## Librerie utilizzate <a id="librerie-utilizzate"></a>

Inizieremo importando le librerie necessarie e preparando l'ambiente di lavoro. Successivamente, analizzeremo il dataset utilizzato per addestrare il modello e lo prenderemo in considerazione per comprendere meglio le caratteristiche dei volti umani. Proseguiremo quindi con la creazione e l'addestramento del Diffusion Model, che utilizzeremo infine per generare nuovi volti.

In [1]:
import torch.nn as nn
from tqdm import tqdm
import logging
from torch.utils.tensorboard import  SummaryWriter
import torch.nn.functional as F
import numpy as np
import os
import copy
import torch
import torchvision
from PIL import Image
import torch.functional
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import random
from torch import optim

ModuleNotFoundError: No module named 'tensorboard'

## Configurazione ambiente <a id="configurazione-ambiente"></a>

Tramite la libreria torch.device viene impostata la scheda video come motore di esecuzione principale per il training della rete.

In [2]:
# Impostazione del seed per la riproducibilità
seed = 33
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")

train_cond = True
batch_size =90
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data augmentation <a id="data-augmentation"></a>

Attraverso la funzione transform vengono applicate delle trasformazioni ad ogni immagine che viene presa in input. Le trasformazioni servono per fare data augmentation, così da poter rendere più facile al modello la creazione di immagini più realistiche.
In particolare vengono applicate:
  - una resize da 64x64 a 80x80;
  - una randomResizedCrop con una dimensione originale di 64x64;
  - un filtro Gaussiano così da permettere alla rete di imapare a riconoscere meglio i volti. Questo potrebbe accadere perché, facendo così, il volto in primo piano risulta più in vista rispetto allo sfondo che apparirebbe più sfuocato;
  - un filtro Gaussiano c;
  - una randomFlip dell'immagine;
  - cambiamento nell'intensità e nella saturazione dei pixel all'interno dell'immagine.

In seguito, grazie ad un dataloader, viene caricato il dataset di immagini con l'applicazione della transformazione.

In [3]:
#uso la trasformazione con i valori di media e deviazione standard tipici della normalizzazione per foto rgb usati nella ResNe
transform = transforms.Compose([
    torchvision.transforms.Resize(80),  # args.image_size + 1/4 *args.image_size
    torchvision.transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.RandomChoice([
    transforms.Lambda(lambda x: torchvision.transforms.GaussianBlur(kernel_size=3, sigma=(0.01, 0.25))(x) if torch.rand(1) < 0.5 else x),]),
    transforms.RandomApply([
        transforms.RandomHorizontalFlip(),
    ], p=0.5),
    transforms.RandomApply([
        transforms.RandomApply([
            transforms.Lambda(lambda x: torchvision.transforms.functional.adjust_hue(x, 0.1)),
            transforms.Lambda(lambda x: torchvision.transforms.functional.adjust_saturation(x, 0.1)),
        ], p=0.2),
    ], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

db = "/kaggle/input/dataset-2-label/dataset_2_label"

dataset = torchvision.datasets.ImageFolder(db, transform=transform)

loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Classe EMA <a id="classe-ema"></a>

La classe "EMA" nel Diffusion Model si riferisce alla Media Mobile Esponenziale. Si tratta di una tecnica utilizzata per calcolare una media ponderata dei dati nel tempo, sostanzialmente viene assegnato un peso maggiore ai dati più recenti.

Permette di mantenere una media mobile dei parametri del modello nel corso dell'addestramento, fornendo una stima più stabile e affidabile dei parametri ottimali. Ciò può aiutare a ridurre l'instabilità durante l'addestramento e migliorare le prestazioni del modello finale. Può essere usata per stabilizzare il processo di generazione facciale producendo immagini realistiche e coerenti nel tempo. Aiuta a ridurre le fluttuazioni indesiderate.

L'EMA è calcolata tramite la seguente formula:

**EMA(t) = (1 - alpha) * EMA(t-1) + alpha * x(t)**

Dove:

- EMA(t) rappresenta il valore dell'EMA al tempo t;
- alpha è un fattore di smoothing che determina il peso relativo dei dati passati rispetto ai nuovi dati. Solitamente, alpha è un valore compreso tra 0 e 1, dove valori più alti danno maggior peso ai dati più recenti;
- EMA(t-1) è il valore dell'EMA al tempo precedente t-1;
- x(t) è il dato corrente al tempo t.


In [4]:
class EMA:
    def __init__(self, alpha):
        super().__init__()
        self.alpha = alpha
        self.step = 0

    def update_model_average(self, ma_model, current_model):
        for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
            old_weight, up_weight = ma_params.data, current_params.data
            ma_params.data = self.update_average(old_weight, up_weight)

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.alpha + (1 - self.alpha) * new

    def step_ema(self, ema_model, model, step_start_ema=2000):
        if self.step < step_start_ema:
            self.reset_parameters(ema_model, model)
            self.step += 1
            return
        self.update_model_average(ema_model, model)
        self.step += 1

    def reset_parameters(self, ema_model, model):
        ema_model.load_state_dict(model.state_dict())

# Calcolo del passo di diffusione <a id="calcolo-del-passo-di-diffusione"></a>

In questo codice viene definita una classe chiamata Diffusion. Questa classe implementa una procedura di diffusione per generare nuove immagini a partire da un modello.

Nel metodo __init__, vengono inizializzati diversi attributi della classe, tra cui noise_steps (numero di passi di rumore), beta_start e beta_end (i valori di inizio e fine per la scheda del rumore), img_size (dimensione delle immagini), e device (dispositivo su cui eseguire il calcolo, di default "cuda" che indica la GPU).

Viene quindi chiamato il metodo prepare_noise_schedule per generare una sequenza di valori di rumore beta distribuiti linearmente tra beta_start e beta_end. Questa sequenza viene spostata sul dispositivo specificato (self.device), e vengono calcolati anche alpha (1 - beta) e alpha_hat che è il cumprod (cumulative product) di alpha.

Il metodo prepare_noise_schedule restituisce una sequenza di valori di rumore beta generati tramite torch.linspace tra beta_start e beta_end con dimensione noise_steps.

Il metodo noise_images prende un tensore x e un indice t, e restituisce un nuovo tensore di rumore generato a partire da x e un rumore casuale Ɛ. Questo avviene calcolando le radici quadrate di alpha_hat e 1 - alpha_hat e moltiplicandole rispettivamente per x e Ɛ. Questo viene fatto per ottenere un contributo di rumore graduale durante il processo di diffusione.

Il metodo sample_timesteps genera un tensore di indici casuali t con dimensione n compresi tra 1 e noise_steps.

Il metodo sample esegue il campionamento di nuove immagini utilizzando il modello specificato. Viene eseguito in modalità di valutazione (model.eval()) e senza calcoli di gradiente (torch.no_grad()). Viene inizializzato un tensore x di rumore casuale. Successivamente, viene iterato all'indietro sui passi di rumore (reversed(range(1, self.noise_steps))) e per ogni passo viene generato il rumore previsto dal modello (predicted_noise) applicato a x. Se cfg_scale è maggiore di 0, viene calcolato anche il rumore previsto in assenza di condizioni (uncond_predicted_noise) e viene eseguita una combinazione lineare tra i due rumori (predicted_noise) in base al valore di cfg_scale. Vengono calcolati i fattori di scala alpha, alpha_hat e beta, e viene generato un rumore casuale noise. Infine, viene applicata la formula di diffusione per aggiornare il tensore x. Alla fine del campionamento, il modello viene riportato in modalità di allenamento (model.train()). Vengono quindi applicate alcune operazioni di normalizzazione e conversione dei valori di x per ottenere un tensore di immagini finale (x).

La formula è composta da
* $\alpha_t = 1 - \beta$
e da
* $\bar{\alpha_t} = \Pi^{t}_{s=1} a_s$
calcolo il primo **semplicemente assegnandolo ad una variabile**, mentre calcolo il secondo usando la **funzione linspace** per calcolare la produttoria.

Vado in seguito a calcolare la formula per il passo di diffusione t restituendolo nella **return**
* $x_t = \sqrt{\bar{a}_t}x_0 + \sqrt{1 - \bar{a}_t}\epsilon$

Eseguo poi il cilo di sampling:

* for $t = T....1$ do
  z - $N(0,1)$ if > 1, else z=0
  $x_{t-1} = \frac{1}{\sqrt{\alpha_t}}(x_t - \frac{1-\alpha_t}{\sqrt{1-\bar{\alpha}}_t}\epsilon_g(x_t,t)) + \sigma_t z$
  end for
  return $x_0$

In [5]:
class Diffusion:
    def __init__(self, noise_steps=400, beta_start=1e-4, beta_end=0.02, img_size=64, device="cuda"):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size
        self.device = device

        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)

    def prepare_noise_schedule(self):
        return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)

    def noise_images(self, x, t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        Ɛ = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ

    def sample_timesteps(self, n):
        return torch.randint(low=1, high=self.noise_steps, size=(n,))

    def sample(self, model, n, labels, cfg_scale=3):
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            x = torch.randn((n, 3, self.img_size, self.img_size)).to(self.device)
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device)
                predicted_noise = model(x, t, labels)
                if cfg_scale > 0:
                    uncond_predicted_noise = model(x, t, None)
                    predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, cfg_scale)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        model.train()
        x = (x.clamp(-1, 1) + 1) / 2
        x = (x * 255).type(torch.uint8)
        return x

# Metodi ausialiari per la creazione della rete <a id="metodi-ausiliari-per-la-creazione-della-rete"></a>

Il metodo __init__ inizializza la classe DoubleConv. Prende diversi argomenti: in_channels (numero dei canali di input), out_channels (numero dei canali di output), mid_channels (numero dei canali intermedi, che di default è uguale a out_channels se non viene fornito), e residual (un flag booleano che indica se utilizzare le connessioni residue, che di default è False).

All'interno del metodo __init__, il flag residual viene assegnato all'attributo self.residual per essere utilizzato successivamente nel metodo forward.

L'attributo double_conv viene definito come un contenitore sequenziale di moduli PyTorch. Consiste in due strati convoluzionali con dimensione del kernel 3 e padding 1, seguiti da normalizzazione di gruppo e attivazione GELU. Il primo strato convoluzionale prende in_channels come input e produce mid_channels come output, mentre il secondo strato convoluzionale prende mid_channels come input e produce out_channels come output. La normalizzazione di gruppo viene applicata dopo ogni strato convoluzionale, con una dimensione di gruppo di 1, il che significa che la normalizzazione viene eseguita indipendentemente per ogni canale.

Nel passo di forward, qualora il flag di blocco residuale sia attivo, viene attivato il passo residuale stesso, altrimenti no.

In [6]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):
        super().__init__()
        self.residual = residual
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, mid_channels),
            nn.GELU(),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, out_channels),
        )

    def forward(self, x):
        if self.residual:
            return F.gelu(x + self.double_conv(x))
        else:
            return self.double_conv(x)

Il modulo SelfAttention implementa un meccanismo di self-attention per elaborare un tensore di input. Applica la self-attention per catturare le relazioni tra le diverse posizioni nel tensore di input e modifica l'input originale utilizzando un modulo di feed-forward per ottenere un output arricchito con informazioni di attenzione.

Nel metodo __init__, vengono inizializzati gli attributi della classe, tra cui channels (numero di canali del tensore di input) e size (dimensione del lato del tensore di input). Viene creato un modulo di multihead attention (self.mha) utilizzando la classe nn.MultiheadAttention. Questo modulo prende in input il numero di canali, il numero di testine di attenzione (impostato a 4) e l'opzione batch_first impostata su True per specificare che la dimensione del batch è la prima dimensione del tensore di input. Viene inoltre creato un modulo di layer normalization (self.ln) utilizzando la classe nn.LayerNorm con una dimensione di input uguale a [channels] per normalizzare i valori lungo la dimensione dei canali.

Il modulo ff_self è definito come un sequenziale di operazioni. Comprende un modulo di layer normalization (nn.LayerNorm) con una dimensione di input uguale a [channels], seguito da due moduli lineari (nn.Linear) con la stessa dimensione di input e output channels, separati da una funzione di attivazione GELU (nn.GELU).

In [7]:
class SelfAttention(nn.Module):
    def __init__(self, channels, size):
        super(SelfAttention, self).__init__()
        self.channels = channels
        self.size = size
        self.mha = nn.MultiheadAttention(channels, 4, batch_first=True)
        self.ln = nn.LayerNorm([channels])
        self.ff_self = nn.Sequential(
            nn.LayerNorm([channels]),
            nn.Linear(channels, channels),
            nn.GELU(),
            nn.Linear(channels, channels),
        )

    def forward(self, x):
        x = x.view(-1, self.channels, self.size * self.size).swapaxes(1, 2)
        x_ln = self.ln(x)
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        attention_value = attention_value + x
        attention_value = self.ff_self(attention_value) + attention_value
        return attention_value.swapaxes(2, 1).view(-1, self.channels, self.size, self.size)

In [9]:
def plot_images(images, num_samples):
    plt.figure(figsize=(32, 32))
    plt.imshow(torch.cat([
        torch.cat([i for i in images[:num_samples].to(device)], dim=-1),], dim=-2).permute(1, 2, 0).cpu())
    plt.show()


def save_images(images, path, **kwargs):
    grid = torchvision.utils.make_grid(images, **kwargs)
    ndarr = grid.permute(1, 2, 0).to('cpu').numpy()
    im = Image.fromarray(ndarr)
    im.save(path)

# Rete Convoluzionale <a id="rete-convoluzionale"></a>

La classe "MyUNetConditioned" è un modulo di rete neurale convoluzionale che implementa un'architettura di rete. Essa è una versione personalizzata di una rete UNet e viene utilizzata per la segmentazione di immagini. Essa presenta 4 blocchi convoluzioniali di discesa, un passo di bottleneck e altri 4 passi convoluzionali di salita inframezzati dal calcolo di layer di selfAttention per poter marcare meglio quelle che sono le caratteristiche che spiccano di più di ogni singola foto.
Abbiamo due variabili, c?in e c?out che indicano i canali in entrata e i canali in uscita della rete. Si parte quindi da un numero di canali pari a 3 per arrivare, all'interno del passo di bottleneck ad un numero di canali di 512. La prima chiamata alla classe DoubleConv contiene semmpre il flag **residual=True**, questo sta ad indicare che per quello specifico blocco convoluzionale è viene usata una rete residuale. Durante il passo di discesa viene utilizzata una funzione "MaxPool2d". La funzione nn.MaxPool2d è un modulo di PyTorch che implementa l'operazione di max pooling su un input bidimensionale. Il max pooling riduce la dimensione spaziale di un tensore di input mantenendo il valore massimo all'interno di ciascuna finestra di pooling.
Nel passo di risalita invece viene utilizzata la funzione Upsample. La funzione nn.Upsample è un modulo di PyTorch che implementa l'operazione di upsampling (aumento della risoluzione) su un input. L'upsampling aumenta le dimensioni spaziali di un tensore interpolando i valori esistenti.

Viene in seguito, definita la funzione di **pos_encoding**. Il metodo pos_encoding definisce la codifica di posizione utilizzata nella classe MyUNetConditioned. Questa codifica di posizione viene applicata al tempo t per incorporare l'informazione temporale nel modello. Questa codifica di posizione basata sul tempo t utilizza il seno e coseno. Essa viene successivamente utilizzata nel metodo forward per arricchire l'input del modello con informazioni temporali.

Il metodo **forward** è responsabile dell'esecuzione dell'inoltro (forward pass) della rete neurale durante l'elaborazione dei dati. Prende in input il tensore x (input dell'immagine), il tempo t e l'etichetta y. t = t.unsqueeze(-1).type(torch.float): Aggiunge una dimensione all'input del tempo t all'ultimo indice utilizzando unsqueeze. Questo è fatto per garantire che il tensore del tempo abbia le stesse dimensioni del tensore di codifica di posizione restituito dal metodo pos_encoding. Successivamente, il tipo del tensore viene convertito in torch.float. Viene in seguito applicato il pos_encoding e si va a controllare.

**if y is not None: t += self.label_emb(y):** Se l'etichetta y è disponibile (non è None), viene applicata un'embedding all'etichetta y utilizzando self.label_emb e poi sommata al tensore t. Questo permette di incorporare l'informazione dell'etichetta nel tensore t.

Viene in seguito applicato il pattern di convoluzioni così come sono state definite e viene ritornato nell'ultima riga con **return self.exit(x4)** il risultatp, che equivale ad un tensore dopo l'applicazione dell'ultima convoluzione finale.


In [8]:
class MyUNetConditioned(nn.Module):
    def __init__(self, c_in=3, c_out=3, time_dim=256,num_classes=None, device="cuda"):
        super().__init__()
        self.time_dim = time_dim
        self.device = device

        self.init = DoubleConv(c_in,64)

        self.maxpool_conv1 = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(64, 64, residual=True),
            DoubleConv(64, 128),
        )
        self.emb_layer1 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_dim,128),
        )
        self.sa1 = nn.Sequential(
            SelfAttention(128,32)
        )

        self.maxpool_conv2 = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(128, 128, residual=True),
            DoubleConv(128, 256),
        )
        self.emb_layer2 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_dim,256),
        )
        self.sa1_1 = nn.Sequential(
            SelfAttention(256,16)
        )

        self.maxpool_conv2_1 = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(256, 256, residual=True),
            DoubleConv(256, 256),
        )
        self.emb_layer2_1 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_dim,256),
        )


        #Bottleneck
        self.bottleneck = nn.Sequential(
            DoubleConv(256, 512),
            DoubleConv(512, 512),
            DoubleConv(512, 256),
        )


        self.up3 = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.up_conv3 = nn.Sequential(
            DoubleConv(512, 512, residual=True),
            DoubleConv(512, 128),
        )
        self.emb_layer3 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_dim,128),
        )
        self.sa2 = nn.Sequential(
            SelfAttention(128,16)
        )

        self.up3_1 = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.up_conv3_1 = nn.Sequential(
            DoubleConv(256, 256, residual=True),
            DoubleConv(256, 64),
        )
        self.emb_layer3_1 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_dim,64),
        )
        self.sa1_2 = nn.Sequential(
            SelfAttention(64,32)
        )

        self.up4 = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.up_conv4 = nn.Sequential(
            DoubleConv(128, 128, residual=True),
            DoubleConv(128, 64),
        )
        self.emb_layer4 = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_dim,64),
        )


        self.exit = nn.Conv2d(64,c_out,1)

        if num_classes is not None:
            self.label_emb = nn.Embedding(num_classes, time_dim)

    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (10000 ** (torch.arange(0, channels, 2, device=self.device).float() / channels))
        pos_enc_a = torch.sin(t.repeat(1, channels // 2) * inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels // 2) * inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc

    def forward(self, x, t, y):
        t = t.unsqueeze(-1).type(torch.float)
        t = self.pos_encoding(t, self.time_dim)

        if y is not None:
            t += self.label_emb(y)

        x = self.init(x)


        x1 = self.maxpool_conv1(x)
        emb1 = self.emb_layer1(t)[:, :, None, None].repeat(1, 1, x1.shape[-2], x1.shape[-1])
        x1 = x1 + emb1
        x1 = self.sa1(x1)


        x2 = self.maxpool_conv2(x1)
        emb2 = self.emb_layer2(t)[:, :, None, None].repeat(1, 1, x2.shape[-2], x2.shape[-1])
        x2 = x2 + emb2
        x2 = self.sa1_1(x2)


        x2_1 = self.maxpool_conv2_1(x2)
        emb2_1 = self.emb_layer2_1(t)[:, :, None, None].repeat(1, 1, x2_1.shape[-2], x2_1.shape[-1])
        x2_1 = x2_1 + emb2_1

        #Bottleneck
        bottleneck = self.bottleneck(x2_1)


        x3 = self.up3(bottleneck)
        x3 = torch.cat((x3, x2), dim=1)
        x3 = self.up_conv3(x3)
        emb3 = self.emb_layer3(t)[:, :, None, None].repeat(1, 1, x3.shape[-2], x3.shape[-1])
        x3 = x3 + emb3
        x3 = self.sa2(x3)


        x3_1 = self.up3_1(x3)
        x3_1 = torch.cat((x3_1, x1), dim=1)
        x3_1 = self.up_conv3_1(x3_1)
        emb3_1 = self.emb_layer3_1(t)[:, :, None, None].repeat(1, 1, x3_1.shape[-2], x3_1.shape[-1])
        x3_1 = x3_1 + emb3_1
        x3_1 = self.sa1_2(x3_1)


        x4 = self.up3_1(x3_1)
        x4 = torch.cat((x4, x), dim=1)
        x4 = self.up_conv4(x4)
        emb4 = self.emb_layer4(t)[:, :, None, None].repeat(1, 1, x4.shape[-2], x4.shape[-1])
        x4 = x4 + emb4


        return self.exit(x4)

![https://i.imgur.com/ip66Gk7.png](https://i.imgur.com/ip66Gk7.png)

La rete segue l’andamento mostrato nella figura. Come si può vedere si parte da una dimensione di 64x64 e si arriva ad una dimensione di 8x8. Inoltre, la rete parte prendendo in input 3 canali in ingresso (RGB) per arrivare a 512 nel passo di bottleneck e restituirne, infine, nuovamente 3.
* In giallo sono mostrati i blocchi di SelfAttention applicati all’output delle singole convoluzioni
* In blu sono mostrate le singole convoluzioni
* In rosso sono mostrati i blocchi di maxpool, in verde quello di upsample

# Training Loop <a id="training-loop"></a>

La funzione train definisce il ciclo di addestramento del modello. Prende in input diversi parametri come run_name (nome dell'esecuzione), epochs (numero di epoche di addestramento), image_size (dimensione delle immagini), device (dispositivo di calcolo), lr (learning rate), num_classes (numero di classi), checkpoint (per riprendere l'addestramento da un checkpoint salvato in precedenza).

Ecco come funziona la funzione train:

* Viene inizializzato il dispositivo di calcolo device utilizzando il valore fornito come input.
* Viene creato un oggetto dataloader utilizzando il modulo loader (presumibilmente un oggetto DataLoader che carica i dati per l'addestramento).
* Viene istanziato il modello MyUNetConditioned specificando il numero di classi num_classes. Il modello viene spostato sul dispositivo di calcolo utilizzando il metodo to(device).
* Se è specificato un checkpoint (checkpoint=True), il modello viene caricato utilizzando model.load_state_dict(torch.load(checkpoint)).
* Viene istanziato l'ottimizzatore AdamW utilizzando i parametri del modello e il learning rate lr.
* Viene definita la loss function mse (Mean Squared Error) utilizzando nn.MSELoss().
* Viene creato un oggetto diffusion utilizzando la classe Diffusion, specificando la dimensione delle immagini image_size e il dispositivo di calcolo device.
* Viene istanziato un oggetto EMA (Exponential Moving Average) utilizzando un fattore di decadimento 0.995.
* Viene creato un modello ema_model utilizzando la funzione copy.deepcopy(model). Il modello viene impostato come valutazione (eval()) e non richiede il calcolo del gradiente (requires_grad_(False)).
* Inizia il ciclo di addestramento delle epoche. Viene iterato su ciascuna epoca.
* Viene creato un oggetto tqdm per visualizzare la barra di avanzamento del training.
* Viene iterato su ogni batch di dati nel dataloader.
* Le immagini e le etichette vengono spostate sul dispositivo di calcolo utilizzando images.to(device) e labels.to(device).
* Viene campionato un tensore di timesteps t utilizzando il metodo sample_timesteps dell'oggetto diffusion.
* Vengono generati x_t (immagini con rumore) e noise utilizzando il metodo noise_images dell'oggetto diffusion passando le immagini di input images e il tensore dei tempi t.
* Se un numero casuale generato è inferiore a 0.1, l'etichetta labels viene impostata come None.
* Viene calcolato il rumore predetto utilizzando il modello model applicando x_t, t e labels.
* Viene calcolata la loss function confrontando il rumore predetto con il rumore reale, utilizzando la funzione mse(noise, predicted_noise).
* Vengono azzerati i gradienti degli ottimizzatori chiamando optimizer.zero_grad().
* Viene calcolato il gradiente della loss function chiamando loss.backward().
* Vengono eseguiti i passaggi di ottimizzazione chiamando optimizer.step().
* Viene eseguito il passo EMA (Exponential Moving Average) chiamando ema.step_ema(ema_model, model).
* Vengono aggiornate le informazioni sulla barra di avanzamento chiamando pbar.set_postfix(MSE=loss.item()).
* Se l'epoca è un multiplo di 100, vengono campionate immagini utilizzando il modello model e l'oggetto diffusion passando il numero di classi num_classes e le immagini vengono salvate in una cartella specifica.
* Vengono salvati i checkpoint del modello ema_model e model in file separati.

Il ciclo di addestramento termina dopo tutte le epoche specificate che nel caso attuale sono 500.

In [10]:
def train(run_name = "DDPM_Condtional",epochs = 500,image_size = 64, device = "cuda",lr = 3e-4,num_classes = 22,checkpoint=False):
    device = device
    dataloader = loader
    model = MyUNetConditioned(num_classes=num_classes).to(device)
        
    if checkpoint:
        model.load_state_dict(torch.load(checkpoint))
        
        print("Loaded checkpoint")
        
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    mse = nn.MSELoss()
    diffusion = Diffusion(img_size=image_size, device=device)
    #logger = SummaryWriter(os.path.join("runs", run_name))
    l = len(dataloader)
    ema = EMA(0.995)
    ema_model = copy.deepcopy(model).eval().requires_grad_(False)
    
    

    for epoch in range(epochs):
        #logging.info(f"Starting epoch {epoch}:")
        pbar = tqdm(dataloader)
        print("Epoch number: ", epoch+1)
        for i, (images, labels) in enumerate(pbar):
            images = images.to(device)
            labels = labels.to(device)
            t = diffusion.sample_timesteps(images.shape[0]).to(device)
            x_t, noise = diffusion.noise_images(images, t)
            if np.random.random() < 0.1:
                labels = None
            predicted_noise = model(x_t, t, labels)
            loss = mse(noise, predicted_noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            ema.step_ema(ema_model, model)
            pbar.set_postfix(MSE=loss.item())
            #logger.add_scalar("MSE", loss.item(), global_step=epoch * l + i)

        if (epoch + 1) % 70 == 0:
            labels = torch.arange(num_classes).long().to(device)
            #sampled_images = diffusion.sample(model, n=len(labels), labels=labels)
            ema_sampled_images = diffusion.sample(ema_model, n=len(labels), labels=labels)
            #plot_images(sampled_images,22)
            #plot_images(ema_sampled_images, 8)
            #save_images(sampled_images, os.path.join("results", "/kaggle/working/", f"{epoch}.jpg"))
            save_images(ema_sampled_images, os.path.join("results_ema", "/kaggle/working/", f"{epoch+1}.jpg"))
        torch.save(ema_model.state_dict(), os.path.join("models", "/kaggle/working/", f"ema_ckpt.pt"))
        torch.save(model.state_dict(), os.path.join("models", "/kaggle/working/", f"ckpt.pt"))

# Main <a id="main"></a>

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = MyUNetConditioned(num_classes = 22)

total_params = count_parameters(model)
print("Total number of parameters: ", total_params)

train(run_name = "DDPM_Uncondtional",
    epochs = 100,
    image_size = 64,
    #dataset_path = r"C:\Users\dome\datasets\landscape_img_folder",
    device = "cuda",
    lr = 3e-4,
    checkpoint=r"/kaggle/input/ema-ckpt-tuning/ema_ckpt.pt"
     )